In [1]:
import re
import json
import requests
import random
from fake_useragent import UserAgent
from celery import Celery
from bs4 import BeautifulSoup
from pymongo import MongoClient
from bson.objectid import ObjectId

In [9]:
def random_header():
    ua = UserAgent()
    random_header = json.loads(r'''{
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Host": "www.dogforum.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent":"%s"
    }'''%ua.random)
    return random_header

In [4]:
url = 'http://www.dogforum.com/dogs/'
resp = requests.get(url, headers=header)
hrefs = [href for href in re.findall("""<a href="(http://www.dogforum.com/dogs/\S*/)">""", resp.text)]

In [5]:
hrefs[0]

'http://www.dogforum.com/dogs/nysha_29425/'

In [6]:
def get_dog_detl(url):
    resp = requests.get(url, headers=random_header())
    soup = BeautifulSoup(resp.text, 'lxml')
    data = {}
    pack = {} 
    try:
        data['pet_name'] = soup.select_one('#name').text
    except (IndexError):
        data['pet_name'] = ""
    try:
        data['owner_url'] = soup.select('table')[13].select('td > a')[0]['href']
    except (IndexError):
        data['owner_url'] = ""
    try:
        data['owner_name'] = soup.select('table')[13].select('td > a')[0].text
    except (IndexError):
        data['owner_name'] = ""
    try:
        data['pet_name'] = soup.select('table')[13].select('#name')[0].text
    except (IndexError):
        data['pet_name'] = ""
    try:
        data['breed'] = re.findall("Breed</h3></td><td>(.{2,30})</td></tr>",str(soup.select('table')[13]))[0]
    except (IndexError):
        data['breed'] = ""
    try:
        data['gender'] = soup.select('table')[13].select('#gender')[0].text
    except (IndexError, KeyError):
        data['gender'] = ""
    try:
        data['age'] = re.findall(">Age</h3></td><td>(.{2,30})</td></tr>",str(soup.select('table')[13]))[0]
    except (IndexError, KeyError):
        data['age'] = ""
    try:
        data['food'] = soup.select('table')[13].select('#food')[0].text
    except (IndexError, KeyError):
        data['food'] = ""
    try:
        data['snacks'] = soup.select('table')[13].select('#snakes')[0].text
    except (IndexError, KeyError):
        data['snacks'] = ""
    try:
        data['toys'] = soup.select('table')[13].select('#toys')[0].text
    except (IndexError, KeyError):
        data['toys'] = ""
    try:
        data['activities'] = soup.select('table')[13].select('#activities')[0].text
    except (IndexError, KeyError):
        data['activities'] = ""
    for num in range(0,20):
        try:
            key = soup.select('table')[14].select('a')[num].text
            value = soup.select('table')[14].select('a')[num]['href']
            pack[key] = value
        except (AttributeError, IndexError, KeyError):
            continue
    try:
        pack.pop('')
        data['pack'] = pack
    except (IndexError, KeyError):
        data['pack'] = ""
    to_mongoDB(data)

In [11]:
def to_mongoDB(data):
    conn = MongoClient('localhost', 27017)
    db = conn.dogs
    collection = db.dog
    results = collection.insert_one(data)
    conn.close()

In [13]:
for url in hrefs:
    get_dog_detl(url)

InvalidDocument: key 'Fat Fat.' must not contain '.'